In [1]:
import torch
from AttentionClass import EncoderSelfAttn as encoder
from data_generation.square_sequence import generate_sequences

import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU
import torch.nn as nn
from torch_geometric.nn import NNConv, Set2Set


In [2]:
#Creating data points (of squares) and their direction
points, directions=generate_sequences(n=128, seed=13)

In [3]:
full_seq = (torch.tensor([[-1, -1], [-1, 1], [1, 1], [1, -1]]).float().view(1, 4, 2))
source_seq = full_seq[:, :2]
target_seq = full_seq[:, 2:]

In [4]:
torch.manual_seed(11)
encself=encoder(n_heads=3, d_model=3,ff_units=10, n_features=2)
encself

EncoderSelfAttn(
  (self_attn_heads): MultiHeadAttention(
    (linear_out): Linear(in_features=9, out_features=3, bias=True)
    (attn_heads): ModuleList(
      (0): Attention(
        (linear_query): Linear(in_features=2, out_features=3, bias=True)
        (linear_keys): Linear(in_features=2, out_features=3, bias=True)
        (linear_value): Linear(in_features=2, out_features=3, bias=True)
      )
      (1): Attention(
        (linear_query): Linear(in_features=2, out_features=3, bias=True)
        (linear_keys): Linear(in_features=2, out_features=3, bias=True)
        (linear_value): Linear(in_features=2, out_features=3, bias=True)
      )
      (2): Attention(
        (linear_query): Linear(in_features=2, out_features=3, bias=True)
        (linear_keys): Linear(in_features=2, out_features=3, bias=True)
        (linear_value): Linear(in_features=2, out_features=3, bias=True)
      )
    )
  )
  (ffn): Sequential(
    (0): Linear(in_features=3, out_features=10, bias=True)
    (1): Re

In [5]:
query=source_seq
source_seq, source_seq.size()

(tensor([[[-1., -1.],
          [-1.,  1.]]]),
 torch.Size([1, 2, 2]))

In [6]:
encoder_states=encself(query)
encoder_states, encoder_states.size()

(tensor([[[ 0.5648, -0.3156,  0.3997],
          [ 0.6179, -0.3191,  0.4463]]], grad_fn=<AddBackward0>),
 torch.Size([1, 2, 3]))

In [7]:
x=torch.load("tensor_x.pt", map_location=torch.device("cpu"))
x.size()

torch.Size([37, 11])

In [8]:
lin0=Linear(11, 64)
nn=Sequential(Linear(5,64), ReLU(), Linear(64, 64*64))
conv=NNConv(64, 64, nn, aggr="mean")

In [9]:
#el buen forward
out=F.relu(lin0(x))
out.size()

torch.Size([37, 64])

In [10]:
#se necesita edge_index y edge_attr 
edge_index=torch.load("tensor_edgeindex.pt", map_location=torch.device("cpu"))
edge_attr=torch.load("tensor_edgeattr.pt", map_location=torch.device("cpu"))

In [11]:
m=conv(out, edge_index, edge_attr)
m.size()

torch.Size([37, 64])

In [12]:
batch=torch.load("tensor_batch.pt", map_location=torch.device("cpu"))

In [13]:
batch

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6,
        6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7])

In [14]:


def batch_sequences_features(x, batch):
    def contador(batch):
        diccionario={}
        for i in batch:
            if i.item() not in diccionario:
                diccionario[i.item()]=1
            else:
                diccionario[i.item()]+=1
        return diccionario

    N_first=[]
    l_inf=0
    l_sup=0
    dictionary=contador(batch)
    for i in dictionary:
        l_sup=l_sup+dictionary[i]
        N_first.append(x[l_inf:l_sup])
        l_inf=l_sup
    return N_first
    
    

In [15]:
new=batch_sequences_features(m, batch)
new[6].size()

torch.Size([8, 64])

In [20]:
from torch.nn.utils.rnn import pad_sequence

padded=pad_sequence(new, batch_first=True)

In [17]:
padded, padded.size()

(tensor([[[-1.7508e-01, -4.4356e-02, -1.3807e-01,  ...,  3.4600e-01,
            2.3789e-01,  7.6430e-02],
          [-3.8973e-01, -2.6293e-01, -7.5908e-01,  ..., -2.1574e-01,
           -8.2913e-02,  8.6494e-02],
          [-3.8973e-01, -2.6293e-01, -7.5909e-01,  ..., -2.1574e-01,
           -8.2914e-02,  8.6497e-02],
          ...,
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00],
          [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  ...,  0.0000e+00,
            0.0000e+00,  0.0000e+00]],
 
         [[ 1.2318e-03, -1.5029e-01, -1.9601e-01,  ...,  3.3872e-01,
            3.9834e-01,  2.3667e-02],
          [-3.2007e-01, -4.0774e-01, -7.3526e-01,  ..., -1.4851e-01,
           -1.6814e-01,  2.3146e-01],
          [-3.2005e-01, -4.0775e-01, -7.3522e-01,  ..., -1.4851e-01,
           -1.6813e-01,  2.3144e-01],
          ...,
    

In [21]:
source_mask=(padded!=0).all(axis=2).unsqueeze(1)

In [19]:
source_mask, source_mask.size()

(tensor([[[ True,  True,  True,  True,  True, False, False, False]],
 
         [[ True,  True,  True,  True, False, False, False, False]],
 
         [[ True,  True,  True, False, False, False, False, False]],
 
         [[ True,  True,  True,  True, False, False, False, False]],
 
         [[ True,  True,  True, False, False, False, False, False]],
 
         [[ True,  True,  True,  True, False, False, False, False]],
 
         [[ True,  True,  True,  True,  True,  True,  True,  True]],
 
         [[ True,  True,  True,  True,  True,  True, False, False]]]),
 torch.Size([8, 1, 8]))

In [20]:
enc=encoder(n_heads=3, d_model=64, ff_units=10, n_features=64)


In [19]:
query=padded
encoder_states=enc(query, source_mask)

NameError: name 'padded' is not defined

In [20]:
encoder_states.size()

torch.Size([1, 2, 3])

In [23]:
t1=torch.tensor([0,1,2,3,4,5,6,7,8,9])
boolean=torch.tensor([1,1,1,1,1,0,0,0,0,0])

In [24]:
boolean=[1==i.item() for i in boolean]
boolean

[True, True, True, True, True, False, False, False, False, False]

In [25]:
#funciona
recuperando=[]
for i in range(len(t1)):
    if boolean[i] ==True:
        recuperando.append(t1[i].item())
    else:
        break

In [26]:
recuperando

[0, 1, 2, 3, 4]

In [18]:
encoder_states

tensor([[[ 0.5648, -0.3156,  0.3997],
         [ 0.6179, -0.3191,  0.4463]]], grad_fn=<AddBackward0>)

In [28]:
encoder_states.size()[2]

64

In [29]:
for i in range(encoder_states.size()[0]):
    for j in range(encoder_states.size()[1]):
        pass

In [30]:
source_mask[2][0][1].item()

True

In [31]:
encoder_states[0][0]

tensor([ 0.1000,  0.0723, -0.0861, -0.2734,  0.2067, -0.2045,  0.2032, -0.2602,
        -0.1351, -0.0769,  0.2075, -0.2780,  0.2169,  0.1598,  0.0793,  0.2469,
        -0.3396, -0.0592,  0.2631, -0.2307, -0.2434, -0.1342, -0.1158, -0.2142,
         0.1749, -0.2383,  0.0433,  0.1051, -0.2272, -0.0422, -0.1591, -0.1499,
        -0.0032,  0.1410, -0.0778, -0.1098,  0.3626, -0.3343, -0.1475, -0.2934,
        -0.1260, -0.2685, -0.0698, -0.2012, -0.2344, -0.2556,  0.1946,  0.0303,
        -0.0981, -0.2796, -0.2141,  0.0466, -0.2246,  0.0156,  0.0392, -0.1657,
        -0.0387,  0.2308, -0.1283,  0.0058,  0.1963, -0.2459,  0.0154,  0.1605],
       grad_fn=<SelectBackward>)

In [39]:
get_back=[]
for i in range(encoder_states.size()[0]):
    for j in range(encoder_states.size()[1]):
        if source_mask[i][0][j]==True:
            get_back.append(torch.as_tensor(encoder_states[i][j]).float())
        else:
            pass
len(get_back)

NameError: name 'source_mask' is not defined

In [41]:
mask

tensor([[[ True,  True,  True,  True,  True, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True,  True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True,  True, False, False]]])

In [34]:
my_tensor=torch.cat([get_back[0].unsqueeze(0),get_back[1].unsqueeze(0), get_back[2].unsqueeze(0)], dim=0)

In [35]:
my_tensor.size()

torch.Size([3, 64])

In [36]:
my_tensor=torch.cat([get_back[i].unsqueeze(0) for i in range(len(get_back))], dim=0)

In [37]:
my_tensor.size()

torch.Size([37, 64])

In [21]:
from FullENcoder import EncoderSelfAttnL

In [22]:
#crear un encoder
Encoder=EncoderSelfAttnL(n_heads=3, d_model=64, ff_units=10, n_features=64)

In [23]:
bf=Encoder.batch_sequences_features(m, batch)
bf

[tensor([[-1.7508e-01, -4.4356e-02, -1.3807e-01,  4.4173e-01,  1.4721e-01,
          -2.9860e-01, -6.1902e-02, -6.6204e-01, -2.7573e-01, -5.5273e-01,
           5.2367e-01, -3.0145e-01, -2.8304e-02, -1.0538e-01,  2.3172e-01,
           1.0384e-01,  9.2456e-01, -1.1666e-01,  3.8690e-02, -6.1394e-01,
          -2.7539e-01, -5.4298e-01, -3.5077e-01, -3.8745e-01, -8.5473e-01,
           6.0003e-02, -6.8451e-01,  5.8620e-01, -9.4079e-01,  2.5301e-01,
           5.2283e-01, -4.3084e-01, -5.8201e-01, -3.3532e-01,  2.0599e-01,
          -3.5850e-01,  6.1321e-01,  6.6185e-01,  4.5957e-02,  1.2202e+00,
          -7.7793e-01,  6.1839e-01,  5.9122e-01, -1.4334e-01, -2.2483e-01,
           5.5341e-02, -2.6994e-01, -5.8220e-01,  3.4837e-01,  3.8001e-01,
          -7.4367e-02, -4.6490e-01,  1.9846e-04, -2.6238e-01,  2.1606e-01,
          -2.1853e-01,  1.0127e-02,  3.7984e-01,  2.7198e-01, -1.2992e-01,
          -4.2453e-01,  3.4600e-01,  2.3789e-01,  7.6430e-02],
         [-3.8973e-01, -2.6293e-01, -

In [16]:
from torch.nn.utils.rnn import pad_sequence
padded=pad_sequence(bf, batch_first=True)


In [17]:
def pad_collate(batch):
    x=[item for item in batch]
    x_pad=pad_sequence(x, batch_first=True)
    return x_pad

In [43]:
padded=pad_collate(bf)

In [44]:
from PdA import pad_collate_1

In [45]:
padded=pad_collate_1(bf)
#padded, padded.size()

In [24]:
def contador(batch):
    diccionario={}
    for i in batch:
        if i.item() not in diccionario:
            diccionario[i.item()]=1
        else:
            diccionario[i.item()]+=1
    return diccionario

In [25]:
cuenta=contador(batch)
cuenta

{0: 5, 1: 4, 2: 3, 3: 4, 4: 3, 5: 4, 6: 8, 7: 6}

In [26]:
max_key=max(cuenta, key=cuenta.get)
cuenta[max_key]

8

In [53]:
all_zeros=zeros.repeat(3,1)
all_zeros

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [50]:
bf[0].size()[1]

64

In [53]:
torch.zeros(5).repeat(2,1)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [27]:
cuenta=cuenta
max_key=max(cuenta, key=cuenta.get)
cuenta[max_key]
dim=64
for i,j in enumerate(bf):
    dif=cuenta[max_key]-len(j)
    #print(dif)
    all_zeros=torch.zeros(dim).repeat(dif,1)
    bf[i]=torch.cat([bf[i], all_zeros],0)
tensor=torch.stack(bf)
tensor.size()

torch.Size([8, 8, 64])

In [87]:
#bf[0]=torch.cat([bf[0], torch.zeros(dim).repeat(3,1)])
#bf[1].size()

In [22]:
py_tensor=torch.stack(bf)

In [28]:
from FullENcoder import EncoderSelfAttnL


In [29]:
encoder2=EncoderSelfAttnL(n_heads=3, d_model=64, ff_units=10, n_features=64)
batch

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6,
        6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7])

In [30]:
bf=encoder2.batch_sequences_features(m, batch)

In [33]:
query, mask=encoder2.my_padding(bf, batch)

In [36]:
query.size()

torch.Size([8, 8, 64])

In [37]:
mask

tensor([[[ True,  True,  True,  True,  True, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True, False, False, False, False, False]],

        [[ True,  True,  True,  True, False, False, False, False]],

        [[ True,  True,  True,  True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True,  True, False, False]]])

In [38]:
source_seq

tensor([[[-1., -1.],
         [-1.,  1.]]])

In [ ]:
def get_back(encoded_states,mask):
    get_back=[]
    for i in range(help1.size()[0]):
        for j in range(help1.size()[1]):
            if mask[i][0][j]==True:
                get_back.append(torch.as_tensor(encoded_states[i][j]).float())
                
            else:
                pass
    return get_back